In [67]:
import pandas as pd
import requests
from io import StringIO
def GetFinancialRatios(stock):
    #https://www.cmoney.tw/finance/f00043.aspx?s=3413
    url = 'https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetFinancialRatios&stockId='+str(stock)+'&selectedType=4&cmkey=JAnsgrhkpgvrmXVY2t4HLA%3D%3D'
    ref = 'https://www.cmoney.tw/finance/f00043.aspx?s='+str(stock)
    
    # 偽瀏覽器，cmoney必需加上Referer header 才能抓取資料
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36','Referer': ref}
    
    # 下載該stock的財務比率表，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'
    
    dfs = pd.read_json(StringIO(r.text), encoding='big-5')

    return dfs

In [138]:
GetFinancialRatios(2884).loc[:,['DateRange','EPS','ROE']]

,DateRange,EPS,ROE
0,201901,0.48,3.19
1,201804,0.35,2.35
2,201803,0.38,2.65
3,201802,0.41,2.91
4,201801,0.44,3.15
5,201704,0.29,2.16
6,201703,0.38,2.82
7,201702,0.36,2.75


In [135]:
dfn = GetFinancialRatios(2330).loc[:,['DateRange','EPS','ROE']]
EPSyear = dfn[(dfn['DateRange']>=201801) & (dfn['DateRange']<=201804)]
dfn

,DateRange,EPS,ROE
0,201902,2.57,4.05
1,201901,2.37,3.59
2,201804,3.85,6.15
3,201803,3.44,5.81
4,201802,2.79,4.67
5,201801,3.46,5.74
6,201704,3.83,6.72
7,201703,3.47,6.49


In [76]:
def GetDividendPolicy(stock):
    #https://goodinfo.tw/StockInfo/StockDividendPolicy.asp?STOCK_ID=3413&SHOW_ROTC=
    #https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetDividendPolicy&stockId=3413&cmkey=hFUOuuGdh5qCajakoruDzQ%3D%3D
    url = 'https://www.cmoney.tw/finance/ashx/mainpage.ashx?action=GetDividendPolicy&stockId='+str(stock)+'&cmkey=dTE5zAXVG16aUpndcsFryw%3D%3D'
    ref = 'https://www.cmoney.tw/finance/f00027.aspx?s='+str(stock)
    
    # 偽瀏覽器，cmoney必需加上Referer header 才能抓取資料
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36','Referer': ref}
    
    # 下載該stock的股利政策，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'
    
    dfs = pd.read_json(StringIO(r.text), encoding='big-5')
    #print(dfs)
    return dfs

In [117]:
GetDividendPolicy(2887)


,CashDividend,Date,ReserveAllotment,SurplusAllotment,TotalDividend,TotalStockDividend
0,0.51,2018,0,0.21,0.72,0.21
1,0.54,2017,0,0.44,0.99,0.44
2,0.53,2016,0,0.43,0.95,0.43
3,0.48,2015,0,0.72,1.21,0.72
4,0.10,2014,0,0.00,0.10,0.00
5,0.43,2013,0,0.99,1.42,0.99
6,0.22,2012,0,0.89,1.11,0.89
7,0.22,2011,0,0.90,1.12,0.90
8,0.23,2010,0,0.70,0.93,0.70
9,0.00,2009,0,1.10,1.10,1.10


In [130]:
def evaluate(stock,year):
    df1=GetDividendPolicy(stock)
    df2=GetFinancialRatios(stock)
    #print(df1)
    Dividend = df1[df1['Date']==year]['TotalDividend']
    #EPSyear =  df2.loc[2:5,'EPS'].sum()
    EPSyear = df2[(df2['DateRange']>=201801) & (df2['DateRange']<=201804)]['EPS'].sum()
    
    test=Dividend/EPSyear
    #近四季EPS
    EPS =  df2.loc[0:3,'EPS'].sum()
    ROE =  df2.loc[0:3,'ROE'].sum()
    #盈餘成長率
    result = ROE*(1-Dividend/EPSyear)
    print('股利總和 = %.2f'%Dividend)
    print('盈餘配發率 = %.3f'%test)
    print('2018年度EPS = %.2f'%EPSyear)
    print('近四季ROE = %.2f'%ROE)
    print('近四季EPS = %.2f'%EPS)
    print('盈餘成長率 = %.2f'%result)

    buyprice = result*0.75*EPS
    sellprice = result*EPS

    print('買進參考價 = %.2f'%buyprice)
    print('賣出參考價 = %.2f'%sellprice)

In [136]:
evaluate(2884,2018)

股利總和 = 1.42
盈餘配發率 = 0.899
2018年度EPS = 1.58
近四季ROE = 11.10
近四季EPS = 1.62
盈餘成長率 = 1.12
買進參考價 = 1.37
賣出參考價 = 1.82
